In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial

from sklearn.metrics import root_mean_squared_error

In [2]:
from proc_v2 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter, scheduler

2025-05-17 18:27:52.625377: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-17 18:27:52.634651: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747474072.644572  165005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747474072.647782  165005 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747474072.655384  165005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

# CV_Predict + OOV_Stacking

In [9]:
from sklearn.model_selection import KFold
kf10 = KFold(10, random_state = 123, shuffle = True)

In [4]:
hparams = {
    'model_params': {'n_estimators': 2000, 'colsample_bytree': 0.9, 'num_leaves': 63, 'learning_rate': 0.03},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}
result = sc.cv_result('lgb1_oof', df_train, kf10, hparams, config, lgb_adapter, rerun = 1, result_proc = partial(sgml.predict_learning_result, df=df_test))
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

(0.05920814022675752, [0.05920814022675752])

In [15]:
models = ['lgb2', 'lgb1', 'xgb1', 'xgb2', 'cb1', 'cb2', 'cb3', 'cb4', 'nn1', 'nn2']
cvs = sc.read_cvs(models)

In [24]:
for k, v in cvs.items():
    print(k)
    sc.cv_result(k + '_oof', df_train, kf10, v['hparams'], config, v['adapter'], rerun = 0, result_proc = partial(sgml.predict_learning_result, df=df_test))

lgb2
lgb1


Fold:   0%|          | 0/10 [00:00<?, ?it/s]

xgb1


Fold:   0%|          | 0/10 [00:00<?, ?it/s]

xgb2


...

Step:   0%|          | 0/5274 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/5274 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/5274 [00:00<?, ?it/s]

## Ensemble 2

In [32]:
df_stk_test = pd.DataFrame(
    np.vstack([
        np.vstack(
            sc.read_cv('{}_oof'.format(i))['model_result']
        ).mean(axis = 0) for i in models
    ]).T, index = df_test.index, columns = models
)
df_stk_test.head()

,lgb2,lgb1,xgb1,xgb2,cb1,cb2,cb3,cb4,nn1,nn2
id,,,,,,,,,,
750000,3.327763,3.336512,3.328040,3.338143,3.337005,3.336012,3.337380,3.337825,3.345275,3.344655
750001,4.693969,4.697417,4.701673,4.695919,4.692039,4.690204,4.689820,4.690462,4.691942,4.691630
750002,4.454941,4.462433,4.476429,4.473222,4.474418,4.476163,4.479962,4.476958,4.478114,4.476422
750003,4.837392,4.832312,4.832466,4.841155,4.839224,4.840938,4.839847,4.841208,4.841146,4.840906
750004,4.337783,4.351208,4.339558,4.336888,4.343682,4.344821,4.344227,4.343878,4.346379,4.346787


In [71]:
df_stk = pd.DataFrame(
    np.vstack([
        sc.read_prd('{}_oof'.format(i))  for i in models
    ]).T, index = df_train.index, columns = models
).join(df_train[target])
df_stk.head()

,lgb2,lgb1,xgb1,xgb2,cb1,cb2,cb3,cb4,nn1,nn2,Calories_Log
id,,,,,,,,,,,
0,4.992861,4.994059,4.994693,4.994401,4.998932,4.995558,4.997983,4.994704,4.999161,4.996139,5.017280
1,3.602351,3.616428,3.606463,3.604777,3.604864,3.609916,3.606707,3.611383,3.608615,3.610202,3.555348
2,3.401765,3.391937,3.410879,3.398068,3.395180,3.403103,3.401189,3.397848,3.402780,3.401582,3.401197
3,4.923707,4.921507,4.939140,4.933720,4.933604,4.935626,4.935205,4.931131,4.938252,4.937521,4.948760
4,4.987112,4.993071,4.986852,4.987290,4.992097,4.988107,4.990967,4.991187,4.990809,4.990964,4.990433


In [41]:
result = sgml.cv(df_stk, kf, {'X_num': models}, config, lr_adapter)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.059332502190314176,
 [0.05909817769942466,
  0.06017451761786655,
  0.05754910444171188,
  0.06050820900225361])

In [42]:
from sklearn.linear_model import LinearRegression
reg_meta_lr = LinearRegression()
reg_meta_lr.fit(df_stk, df_train[target])

LinearRegression()

In [43]:
pd.Series(
    reg_meta_lr.predict(df_stk_test), index = df_stk_test.index, name = 'Calories'
).pipe(lambda x: (np.exp(x) - 1).clip(lower = 1)).to_csv('result/submission2.csv')

In [44]:
# !kaggle competitions submit -c playground-series-s5e5 -f result/submission2.csv -m "Ensemble2"

100%|██████████████████████████████████████| 6.06M/6.06M [00:02<00:00, 2.16MB/s]
Successfully submitted to Predict Calorie Expenditure

In [49]:
pd.concat([
    pd.read_csv('result/submission1.csv', index_col = 'id'),
    pd.read_csv('result/submission2.csv', index_col = 'id')
], axis =1).mean(axis = 1).to_csv('result/submission3.csv')

In [50]:
# !kaggle competitions submit -c playground-series-s5e5 -f result/submission3.csv -m "Ensemble3"

100%|██████████████████████████████████████| 6.06M/6.06M [00:06<00:00, 1.04MB/s]
Successfully submitted to Predict Calorie Expenditure

In [52]:
root_mean_squared_error(
    df_train[target], df_stk.mean(axis = 1)
)

0.05924755505412332

In [58]:
df_stk_test.mean(axis = 1).pipe(lambda x: (np.exp(x) - 1).clip(lower = 1)).rename('Calories').to_csv('result/submission4.csv')

In [59]:
# !kaggle competitions submit -c playground-series-s5e5 -f result/submission4.csv -m "Ensemble4" 

100%|██████████████████████████████████████| 6.06M/6.06M [00:06<00:00, 1.02MB/s]
Successfully submitted to Predict Calorie Expenditure

In [85]:
import sgfs

In [83]:
sgfs.step_fs_fast(df_stk.assign(const = 1), models, target, [], set(), [np.inf], root_mean_squared_error)

(['cb1', 'nn2', 'lgb1', 'cb2', 'xgb2', 'cb3', 'lgb2'],
 set(),
 [inf,
  0.05947225212457043,
  0.059275810302857876,
  0.0592055757476724,
  0.05919741016414064,
  0.05919298831611201,
  0.059190882479861526,
  0.05918911022366333])

In [84]:
result = sgml.cv(df_stk, kf, {'X_num': ['cb1', 'nn2', 'lgb1', 'cb2', 'xgb2', 'cb3', 'lgb2']}, config, lr_adapter)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.059266503256573455,
 [0.05905972552454461,
  0.06020766123712949,
  0.057436020144312754,
  0.06036260612030697])